Il k-nearest neighbors è un algoritmo di classificazione basato sull'assunto che le osservazioni simili appartengano alla stessa classe. Consiste nel calcolare la distanza tra un'osservazione di test e tutte le osservazioni del dataset di training e nell'assegnare l'etichetta della classe più presente tra i k osservazioni più vicine.

Il processo è questo:

1) Scegliere il numero di vicini k da considerare durante il calcolo delle distanze

2) Calcolare la distanza tra l'osservazione di test e tutte le osservazioni del dataset di training

3) Selezionare le k osservazioni più vicine

4) Assegnare l'etichetta della classe più presente tra le k osservazioni più vicine all'osservazione di test

Questo algoritmo può essere lento nel caso di dataset di grandi dimensioni e l'accuratezza del modello può essere influenzata in modo significativo dalla scelta del numero di vicini k.


Pro: 

Semplicità: è un algoritmo semplice e facile da capire e implementare

Buona prestazione con dataset di piccole dimensioni

Contro:

Lentezza con dataset di grandi dimensioni perchè deve calcolare la distanza tra l'osservazione di test e tutte le osservazioni ogni volta. Infatti è considerato un algoritmo lazy learner, ossia un algoritmo che richiede meno tempo per l'apprendimento e più tempo per la classificazione

Sensibilità alla scelta del numero k di vicini

Sensibilità alla presenza di outliers

In [66]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")
df <- df_finale

# Feature selection

Feature selection fatta attraverso l'algoritmo boruta che si basa sull'algoritmo di random forest e utilizza una tecnica di confronto dei risultati ottenuti con un insieme di dati di controllo per determinare quali variabili sono significative. è un algoritmo che tiene conto delle interazioni tra variabili, è versatile in quanto lavora sia su variabili continue che categoriche ed è resistente agli outliers

In [67]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +1.7 mins: 

 confirmed 11 attributes: albumina, centreCode, degenzaPreTra, intermedLactates, IntraoperativTransfected and 6 more;

 rejected 41 attributes: age, anastomTechnic, bileAnastomosis, clifSofaForCardio, coldIschemia and 36 more;

 still have 25 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +2 mins: 

 confirmed 2 attributes: platCountTranspl, ratioPaO2FiO2;

 rejected 

 [1] "centreCode"               "degenzaPreTra"           
 [3] "presentRefractAscites"    "nutritStatus"            
 [5] "Sodiemia"                 "ratioPaO2FiO2"           
 [7] "platCountTranspl"         "albumina"                
 [9] "surgeryDuration"          "IntraoperativTransfected"
[11] "plasmaFresco"             "plasmaPools"             
[13] "CRIOprecipitated"         "CristalloidiComp"        
[15] "Albumina50Comp"           "noradrenalineEndSurger"  
[17] "intermedLactates"         "syndromeReperfus"        
[19] "abdomOpenEndSurg"         "scoreCHILD"              
[21] "organDonorDrawalSodiemia" "realMELD"                
[23] "comorbidities_none_RIC"   "new_admWard"             
[25] "EPScum"                   "tumore"                  


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [68]:
plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")

In [69]:
df_boruta$deg_cat <- df$deg_cat

## Data partition

Viene effettuata divisione tra training e test set, che è del 70-30 per tutti i modelli utilizzati

In [70]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)



Viene effettuato un bilanciamento della variabile target perchè in origine la classe ottimo conteneva circa 1/4 dei record rispetto alle altre due classi e i modelli facevano fatica a classificare quel determinato valore. La funzione utilizzata è SmoteClassif() che:

1) Seleziona a caso una osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

2) Seleziona a caso un'altra osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

3) Genera una nuova osservazione sintetica che si trova a metà strada tra le due osservazioni selezionate in precedenza

4) Ripete i passaggi precedenti fino a raggiungere il bilanciamento desiderato


Questa funzione utilizza il knn per calcolare le distanze tra le osservazioni e generare le osservazioni, infatti il valore di k è stato selezionato pari alla radice quadrata del numero totale di record presenti nel training set come si fa per il knn normalmente per il knn



In [71]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

Viene effettuato lo stesso processo considerando il dataset dopo la boruta feature selection invece del dataset con tutte le feature in modo da poter fare una comparazione sui risultati successivamente

In [72]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_boruta$deg_cat, SplitRatio = 0.7)
trainB  <- subset(df_boruta, sample == TRUE)
testB   <- subset(df_boruta, sample == FALSE)



TestClassesB <- testB$deg_cat
TestDataB <- subset(testB,select= -deg_cat)


In [73]:
trainB <- SmoteClassif(deg_cat ~ ., trainB, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(trainB$deg_cat)

TrainClassesB <- trainB$deg_cat
TrainDataB <- subset(trainB,select= -deg_cat)


   fast NotFast 
    401     402 

## Models

Attraverso la funzione trainControl viene effettuata una repeated cross validation e una random search.

La prima permette di ottenere una stima più accurata dell'accuratezza del modello:

1) Dividere il dataset in un numero predeterminato di folds (in questo caso è stato scelto n=10 quindi 10 folds)

2) Eseguire la cross-validation ripetutamente utilizzando ogni fold come test set e gli altri folds come training set

3) Calcolare la media e la deviazione standard delle misure di accuratezza ottenute per ogni fold

4) Valutare l'accuratezza e la robustezza del modello utilizzando la media e la deviazione standard delle misure di accuratezza

L'utilizzo della repeated cross validation è particolarmente importante quando si lavora con dataset di dimensioni ridotte o con un numero limitato di osservazioni perchè in questi casi i risultati possono essere influenzata dall'aleatorietà nella divisione dei dati in training set e test set. La repeated cross-validation permette quindi di ottenere una stima più accurata dell'accuratezza del modello ripetendo il processo  più volte e considerando dati diversi



La random search, invece è una tecnica di ottimizzazione dei modelli che viene utilizzata per selezionare i migliori parametri di un modello. Consiste nel generare a caso diverse combinazioni di parametri e nel valutare l'accuratezza del modello per ogni combinazione. La combinazione di parametri che ottiene il risultato migliore viene quindi selezionata come quella ottimale. Questa soluzione è stata preferita alla grid search che poteva dare risultati migliori per una questione di costo computazionale

In [74]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

Viene allenato il modello knn sul dataset senza feature selection

In [75]:
model_knn <- train(x=TrainData, y=TrainClasses, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
- Fold01.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
+ Fold01.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
- Fold01.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
+ Fold01.Rep1: kmax= 15, distance=0.8723, kernel=inv 
- Fold01.Rep1: kmax= 15, distance=0.8723, kernel=inv 
+ Fold02.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
- Fold02.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
+ Fold02.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
- Fold02.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
+ Fold02.Rep1: kmax= 15, distance=0.8723, kernel=inv 
- Fold02.Rep1: kmax= 15, distance=0.8723, kernel=inv 
+ Fold03.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
- Fold03.Rep1: kmax= 59, distance=1.2490, kernel=rectangular 
+ Fold03.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
- Fold03.Rep1: kmax=239, distance=2.8132, kernel=rectangular 
+ Fold03.Rep1: kmax= 15, distance=0.8723

Viene allenato il modello knn sul dataset con feature selection

In [76]:
model_knnB <- train(x=TrainDataB, y=TrainClassesB, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
- Fold01.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
+ Fold01.Rep1: kmax=199, distance=1.7299, kernel=cos 
- Fold01.Rep1: kmax=199, distance=1.7299, kernel=cos 
+ Fold01.Rep1: kmax=127, distance=2.8932, kernel=inv 
- Fold01.Rep1: kmax=127, distance=2.8932, kernel=inv 
+ Fold02.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
- Fold02.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
+ Fold02.Rep1: kmax=199, distance=1.7299, kernel=cos 
- Fold02.Rep1: kmax=199, distance=1.7299, kernel=cos 
+ Fold02.Rep1: kmax=127, distance=2.8932, kernel=inv 
- Fold02.Rep1: kmax=127, distance=2.8932, kernel=inv 
+ Fold03.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
- Fold03.Rep1: kmax= 83, distance=0.5587, kernel=epanechnikov 
+ Fold03.Rep1: kmax=199, distance=1.7299, kernel=cos 
- Fold03.Rep1: kmax=199, distance=1.7299, kernel=cos 
+ Fold03.Rep1: kmax=127, distance=2.8932, kernel=inv 
- Fold03.Rep1: kmax=127, dis

I parametri testati sono: 

kmax = numero di nearest neighbors considerati

distance = parametro per la distanza minkowski

kernel = per specificare il metodo di con cui vengono calcolati i pesi da utilizzare durante il calcolo delle distanze tra le osservazioni

## Results

Vengono analizzati i risultati ottenuti dai due modelli durante la cross validation. Le metriche utilizzate sono: accuracy e kappa. La prima si riferisce alla percentuale di osservazioni correttamente classificate dal modello rispetto al totale di osservazioni. La seconda viene utilizzata per misurare l'accordo tra più modelli di classificazione. Può assumere valori compresi tra -1 e 1. Un valore di kappa vicino a 1 indica un alto livello di accordo tra le classificazioni, mentre un valore di kappa vicino a 0 indica un accordo simile a quello ottenuto per caso. Un valore di kappa negativo indica un accordo peggiore di quello ottenuto per caso.

In [77]:
print(model_knn)
plot(model_knn)

k-Nearest Neighbors 

803 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 723, 721, 723, 723, 723, 723, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel       Accuracy   Kappa    
   15   0.8723498  inv          0.7150068  0.4304852
   59   1.2489534  rectangular  0.7100369  0.4204686
  239   2.8131649  rectangular  0.6900207  0.3802699

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 15, distance = 0.8723498
 and kernel = inv.


In [78]:
print(model_knnB)
plot(model_knnB)

k-Nearest Neighbors 

803 samples
 26 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 723, 723, 722, 723, 723, 722, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel        Accuracy   Kappa    
   83   0.5587345  epanechnikov  0.7511420  0.5024051
  127   2.8931854  inv           0.7498148  0.4994280
  199   1.7298599  cos           0.7485957  0.4971438

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 83, distance = 0.5587345
 and kernel = epanechnikov.


Vengono messi a confronto i risultati ottenuti sulla cross validation dei due modelli in modo da selezionare il più performante da utilizzare nel test set

In [79]:
# collect resamples
results <- resamples(list(knn=model_knn,knn_boruta=model_knnB))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: knn, knn_boruta 
Number of resamples: 10 

Accuracy 
                Min.  1st Qu.  Median      Mean  3rd Qu.  Max. NA's
knn        0.5555556 0.714253 0.72500 0.7150068 0.746875 0.775    0
knn_boruta 0.6666667 0.721412 0.74375 0.7511420 0.784375 0.825    0

Kappa 
                Min.   1st Qu. Median      Mean 3rd Qu. Max. NA's
knn        0.1158278 0.4285061 0.4500 0.4304852 0.49375 0.55    0
knn_boruta 0.3313971 0.4444409 0.4875 0.5024051 0.56875 0.65    0


## Prediction

Viene selezionato il modello più performante per usarlo nel test set ed avere risultati definitivi.

In [80]:
pred_knn <- predict(model_knnB,test)  
pred_knn.prob <- predict(model_knnB,test, type="prob")


result <- confusionMatrix(TestClasses,  pred_knn) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      92      29
   NotFast   90     133
                                          
               Accuracy : 0.6541          
                 95% CI : (0.6012, 0.7043)
    No Information Rate : 0.5291          
    P-Value [Acc > NIR] : 1.780e-06       
                                          
                  Kappa : 0.3199          
                                          
 Mcnemar's Test P-Value : 3.794e-08       
                                          
            Sensitivity : 0.5055          
            Specificity : 0.8210          
         Pos Pred Value : 0.7603          
         Neg Pred Value : 0.5964          
             Prevalence : 0.5291          
         Detection Rate : 0.2674          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6632          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5054945            0.8209877            0.7603306 
      Neg Pred Value            Precision               Recall 
           0.5964126            0.7603306            0.5054945 
                  F1           Prevalence       Detection Rate 
           0.6072607            0.5290698            0.2674419 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6632411

In [81]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_knn))
roc_knn



Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6784

In [82]:
roc_obj <- roc(TestClasses, as.numeric(pred_knn))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [83]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) in 121 controls (TestClasses fast) < 223 cases (TestClasses NotFast).
Area under the curve: 0.6784

Probabilità per ogni classe

In [84]:
pred_knn.prob

fast,NotFast
0.30518109,0.6948189125
0.07338068,0.9266193213
0.89303487,0.1069651281
0.00000000,1.0000000000
0.75608129,0.2439187105
0.17063186,0.8293681375
0.68017331,0.3198266857
0.88602625,0.1139737550
0.85512588,0.1448741174
0.10547013,0.8945298722
